Task 1.5.3 (role-based access control (rbac) configuration)

Step 1: Setup the Project

In [ ]:
mkdir consultancy_website
cd consultancy_website
python3 -m venv venv
source venv/bin/activate
pip install Flask Flask-SQLAlchemy Flask-Login


Step 2: Define Your Models

In [ ]:
# app.py
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'
app.config['SECRET_KEY'] = 'your_secret_key'
db = SQLAlchemy(app)
login_manager = LoginManager(app)

class Role(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), unique=True, nullable=False)
    users = db.relationship('User', backref='role', lazy=True)

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)
    role_id = db.Column(db.Integer, db.ForeignKey('role.id'), nullable=False)

# Initialize database
with app.app_context():
    db.create_all()


Step 3: User Authentication

In [ ]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        
        if user and user.password == password:  # Implement hashed password in production!
            login_user(user)
            return redirect('/dashboard')
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect('/login')


Step 4: Implement Role-Based Access Control

In [ ]:
from functools import wraps
from flask import redirect, url_for, flash

def role_required(role):
    def wrapper(fn):
        @wraps(fn)
        def inner(*args, **kwargs):
            if current_user.is_authenticated and current_user.role.name == role:
                return fn(*args, **kwargs)
            else:
                flash('You do not have permission to access this page.')
                return redirect(url_for('home'))
        return inner
    return wrapper

@app.route('/admin')
@login_required
@role_required('Admin')
def admin_dashboard():
    return render_template('admin.html')

@app.route('/user')
@login_required
@role_required('User')
def user_dashboard():
    return render_template('user.html')


Step 6: Run the Application

In [ ]:
# Create admin and user roles
admin_role = Role(name='Admin')
user_role = Role(name='User')
db.session.add(admin_role)
db.session.add(user_role)
db.session.commit()

# Create users
admin_user = User(username='admin', password='admin_password', role=admin_role)
regular_user = User(username='user', password='user_password', role=user_role)
db.session.add(admin_user)
db.session.add(regular_user)
db.session.commit()
